In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [23]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_values = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_values:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [25]:
df.head().T

0             1               2  \
customerid              7590-vhveg    5575-gnvde      3668-qpybk   
gender                      female          male            male   
seniorcitizen                    0             0               0   
partner                        yes            no              no   
dependents                      no            no              no   
tenure                           1            34               2   
phoneservice                    no           yes             yes   
multiplelines     no_phone_service            no              no   
internetservice                dsl           dsl             dsl   
onlinesecurity                  no           yes             yes   
onlinebackup                   yes            no             yes   
deviceprotection                no           yes              no   
techsupport                     no            no              no   
streamingtv                     no            no              no   
streamingmovies                 no            no              no   
contract            month-to-month      one_year  month-to-month   
paperlessbilling               yes            no             yes   
paymentmethod     electronic_check  mailed_check    mailed_check   
monthlycharges               29.85         56.95           53.85   
totalcharges                 29.85        1889.5          108.15   
churn                           no            no             yes   

                                          3                 4  
customerid                       7795-cfocw        9237-hqitu  
gender                                 male            female  
seniorcitizen                             0                 0  
partner                                  no                no  
dependents                               no                no  
tenure                                   45                 2  
phoneservice                             no               yes  
multiplelines              no_phone_service                no  
internetservice                         dsl       fiber_optic  
onlinesecurity                          yes                no  
onlinebackup                             no                no  
deviceprotection                        yes                no  
techsupport                             yes                no  
streamingtv                              no                no  
streamingmovies                          no                no  
contract                           one_year    month-to-month  
paperlessbilling                         no               yes  
paymentmethod     bank_transfer_(automatic)  electronic_check  
monthlycharges                         42.3              70.7  
totalcharges                        1840.75            151.65  
churn                                    no               yes

In [26]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [29]:
tc = pd.to_numeric(df.totalcharges, errors='coerce')

In [32]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [33]:
df.totalcharges = df.totalcharges.fillna(0)

In [34]:
df[tc.isnull()][['customerid', 'totalcharges']]

customerid  totalcharges
488   4472-lvygi           0.0
753   3115-czmzd           0.0
936   5709-lvoeq           0.0
1082  4367-nuyao           0.0
1340  1371-dwpaz           0.0
3331  7644-omvmy           0.0
3826  3213-vvolg           0.0
4380  2520-sgtta           0.0
5218  2923-arzlg           0.0
6670  4075-wkniu           0.0
6754  2775-sefee           0.0

In [42]:
df.churn = (df.churn == 'yes').astype(int)

## Setting up validation framework

In [43]:
from sklearn.model_selection import train_test_split

In [55]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [56]:
len(df_full_train), len(df_test)

(5634, 1409)

In [57]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [58]:
len(df_train), len(df_val), len(df_test)

(4225, 1409, 1409)

In [59]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [60]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [61]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

## EDA - Exploretary data anlysis 

In [75]:
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [77]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

In [79]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [81]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [82]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [84]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## Churn rate, risk ratio

In [85]:
df_full_train.head()

customerid  gender  seniorcitizen partner dependents  tenure phoneservice  \
0  5442-pptjy    male              0     yes        yes      12          yes   
1  6261-rcvns  female              0      no         no      42          yes   
2  2176-osjuv    male              0     yes         no      71          yes   
3  6161-erdgd    male              0     yes        yes      71          yes   
4  2364-ufrom    male              0      no         no      30          yes   

  multiplelines internetservice       onlinesecurity  ...  \
0            no              no  no_internet_service  ...   
1            no             dsl                  yes  ...   
2           yes             dsl                  yes  ...   
3           yes             dsl                  yes  ...   
4            no             dsl                  yes  ...   

      deviceprotection          techsupport          streamingtv  \
0  no_internet_service  no_internet_service  no_internet_service   
1                  yes                  yes                   no   
2                   no                  yes                   no   
3                  yes                  yes                  yes   
4                   no                  yes                  yes   

       streamingmovies  contract paperlessbilling              paymentmethod  \
0  no_internet_service  two_year               no               mailed_check   
1                  yes  one_year               no    credit_card_(automatic)   
2                   no  two_year               no  bank_transfer_(automatic)   
3                  yes  one_year               no           electronic_check   
4                   no  one_year               no           electronic_check   

  monthlycharges  totalcharges  churn  
0          19.70        258.35      0  
1          73.90       3160.55      1  
2          65.15       4681.75      0  
3          85.45       6300.85      0  
4          70.40       2044.75      0  

[5 rows x 21 columns]

In [86]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [87]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [88]:
global_churn = df_full_train.churn.mean()
global_churn

0.26996805111821087

In [89]:
df_full_train.partner.value_counts()

no     2932
yes    2702
Name: partner, dtype: int64

In [91]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

0.20503330866025166

In [93]:
global_churn - churn_partner

0.06493474245795922

In [92]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

In [94]:
global_churn - churn_no_partner

-0.05984095297455855

## Risk ratio

In [95]:
churn_no_partner / global_churn

1.2216593879412643

In [96]:
churn_partner / global_churn

0.7594724924338315

In [105]:
from IPython.display import display

In [106]:
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


mean  count      diff      risk
gender                                     
female  0.276824   2796  0.006856  1.025396
male    0.263214   2838 -0.006755  0.974980



seniorcitizen


mean  count      diff      risk
seniorcitizen                                     
0              0.242270   4722 -0.027698  0.897403
1              0.413377    912  0.143409  1.531208



partner


...

mean  count      diff      risk
contract                                           
month-to-month  0.431701   3104  0.161733  1.599082
one_year        0.120573   1186 -0.149395  0.446621
two_year        0.028274   1344 -0.241694  0.104730



paperlessbilling


mean  count      diff      risk
paperlessbilling                                     
no                0.172071   2313 -0.097897  0.637375
yes               0.338151   3321  0.068183  1.252560



paymentmethod


mean  count      diff      risk
paymentmethod                                                 
bank_transfer_(automatic)  0.168171   1219 -0.101797  0.622928
credit_card_(automatic)    0.164339   1217 -0.105630  0.608733
electronic_check           0.455890   1893  0.185922  1.688682
mailed_check               0.193870   1305 -0.076098  0.718121

## Mutual information

In [107]:
from sklearn.metrics import mutual_info_score

In [108]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

0.0983203874041556

In [109]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [112]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## Correlation

In [114]:
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [115]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5953420669577875

In [116]:
df_full_train[df_full_train.tenure > 2].churn.mean()

0.22478269658378816

## One-hot encoding

In [118]:
from sklearn.feature_extraction import DictVectorizer

In [121]:
df_train[['gender', 'contract']].iloc[:10]

gender        contract
0  female        two_year
1    male  month-to-month
2  female  month-to-month
3  female  month-to-month
4  female        two_year
5    male  month-to-month
6    male  month-to-month
7  female  month-to-month
8  female        two_year
9  female  month-to-month

In [135]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [136]:
dv = DictVectorizer(sparse=False)

In [140]:
X_train = dv.fit_transform(train_dicts)

In [142]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [143]:
X_val = dv.transform(val_dicts)

## Logistic regression

In [144]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [145]:
z = np.linspace(-5, 5, 51)
z

array([-5. , -4.8, -4.6, -4.4, -4.2, -4. , -3.8, -3.6, -3.4, -3.2, -3. ,
       -2.8, -2.6, -2.4, -2.2, -2. , -1.8, -1.6, -1.4, -1.2, -1. , -0.8,
       -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,
        1.6,  1.8,  2. ,  2.2,  2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,
        3.8,  4. ,  4.2,  4.4,  4.6,  4.8,  5. ])

In [147]:
plt.plot(z, sigmoid(z))

## Train logistic regression

In [148]:
from sklearn.linear_model import LogisticRegression

In [149]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [152]:
model.intercept_[0]

-0.10900244057223585

In [151]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.317, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.071, -0.059,  0.141,
       -0.249,  0.216, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [156]:
y_pred = model.predict_proba(X_val)[:, 1]

In [162]:
churn_decision = (y_pred >= 0.5)

In [166]:
(y_val == churn_decision).mean()

0.8034066713981547

In [167]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = churn_decision.astype(int)
df_pred['actual'] = y_val

In [168]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [170]:
df_pred.correct.mean()

0.8034066713981547

## Model interpretation

In [171]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

{'contract=month-to-month': 0.475,
 'contract=one_year': -0.175,
 'contract=two_year': -0.408,
 'dependents=no': -0.03,
 'dependents=yes': -0.078,
 'deviceprotection=no': 0.063,
 'deviceprotection=no_internet_service': -0.089,
 'deviceprotection=yes': -0.081,
 'gender=female': -0.034,
 'gender=male': -0.073,
 'internetservice=dsl': -0.335,
 'internetservice=fiber_optic': 0.317,
 'internetservice=no': -0.089,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.258,
 'multiplelines=no_phone_service': 0.141,
 'multiplelines=yes': 0.009,
 'onlinebackup=no': 0.063,
 'onlinebackup=no_internet_service': -0.089,
 'onlinebackup=yes': -0.081,
 'onlinesecurity=no': 0.266,
 'onlinesecurity=no_internet_service': -0.089,
 'onlinesecurity=yes': -0.284,
 'paperlessbilling=no': -0.231,
 'paperlessbilling=yes': 0.124,
 'partner=no': -0.166,
 'partner=yes': 0.058,
 'paymentmethod=bank_transfer_(automatic)': -0.087,
 'paymentmethod=credit_card_(automatic)': -0.032,
 'paymentmethod=electronic_check': 0.071,


In [173]:
small = ['contract', 'tenure', 'monthlycharges']

In [175]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [176]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [177]:
dv_small.get_feature_names()

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'monthlycharges',
 'tenure']

In [178]:
X_train_small = dv_small.transform(dicts_train_small)

In [179]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [181]:
w0 = model_small.intercept_[0]
w0

-2.4767756600387165

In [182]:
w = model_small.coef_[0]
w.round(3)

array([ 0.97 , -0.025, -0.949,  0.027, -0.036])

In [183]:
dict(zip(dv_small.get_feature_names(), w.round(3)))

{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

## Using the model

In [184]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [187]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [188]:
y_full_train = df_full_train.churn.values

In [189]:
model = LogisticRegression()
model.fit(X_full_train, y_full_train)

LogisticRegression()

In [190]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [192]:
X_test = dv.transform(dicts_test)

In [193]:
y_pred = model.predict_proba(X_test)[:, 1]

In [194]:
churn_decision = (y_pred >= 0.5)

In [195]:
(churn_decision == y_test).mean()

0.815471965933286

In [206]:
customer = dicts_test[-1]
customer

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'no',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 17,
 'monthlycharges': 104.2,
 'totalcharges': 1743.5}

In [207]:
X_small = dv.transform([customer])

In [208]:
model.predict_proba(X_small)[0, 1]

0.5968852088419566

In [209]:
y_test[-1]

1